# Test reading a set of rainfields netCDF rain_rate files and writing them to the mongo database

In [1]:
import rain_base_fs
import datetime
import os

In [4]:
# traverse the file directories and write the data out to the database 
start_date = datetime.datetime(2021,1,28) 
end_date = datetime.datetime(2021,1,30)
time_step = datetime.timedelta(minutes=5)
base_dir = "/home/awseed/data/RF3/prcp-crate"
product = "prcp-crate.nc"
stn_id = 310 
start_time = datetime.datetime.now() 

file_number = 0 
vtime = start_date
while vtime < end_date:
    y = vtime.strftime("%Y") 
    ymd = vtime.strftime("%Y%m%d")
    hms = vtime.strftime("%H%M%S")
    file_path= os.path.join(base_dir,y)
    file_path = os.path.join(file_path,ymd)
    file_name = f"{stn_id}_{ymd}_{hms}.{product}"
    file_path = os.path.join(file_path, file_name)
    file_id = rain_base_fs.write_to_rain_basefs(file_path)
    vtime = vtime + time_step  
    file_number += 1

end_time = datetime.datetime.now() 
dt = end_time - start_time 
print(f"Processed {file_number} files in {dt}")

Error: /home/awseed/data/RF3/prcp-crate/2021/20210129/310_20210129_165000.prcp-crate.nc not found
Processed {file_number} files in {dt}
